In [1]:
import os
import numpy as np

from itertools import combinations
from scipy.spatial import cKDTree
from IPython.display import display
import pandas as pd
from math import sqrt

from sklearn.cluster import DBSCAN
import cv2
import networkx as nx
from collections import Counter
import random

import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.csgraph import connected_components
from collections import Counter

import matplotlib.pyplot as plt
import matplotlib.collections as mc
from matplotlib.colors import ListedColormap
import matplotlib
import matplotlib.colors as mcolors
cmap = ListedColormap(['grey', 'red'])
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import colorsys

from sklearn.metrics import adjusted_rand_score, v_measure_score

from matplotlib.path import Path
from matplotlib.patches import Polygon as MatplotlibPolygon

import json


DATA_PATH = "/mnt/cai-data/manuscript-annotation-tool/manuscripts/eeml-submission/points-2D"
IMG_PATH = "/mnt/cai-data/manuscript-annotation-tool/manuscripts/eeml-submission/leaves"
SEAMFORMER_RESULTS = "/home/kartik/layout-analysis/seamformer/seamformer/data/output/eeml-data/v0.json"



## our method

In [2]:
NUM_NEIGHBOURS = 8
cos_similarity_less_than = -0.8
%matplotlib widget


# HELPER FUNCTIONS
def convert_to_graph(to_plot, points):
    # Create an empty graph
    G = nx.Graph()
    
    # Add all points as nodes with their coordinates as features
    for i, point in enumerate(points):
        G.add_node(i, pos=point)
    
    # Add edges from to_plot, avoiding duplicates (NetworkX handles this automatically)
    for center, neighbor1, neighbor2 in to_plot:
        G.add_edge(center, neighbor1)
        G.add_edge(center, neighbor2)
    
    # Assign labels to nodes based on connected components:
    # Every connected component gets a unique label.
    for label, component in enumerate(nx.connected_components(G)):
        for node in component:
            G.nodes[node]['label'] = label
    
    # Create adjacency matrix
    adj_matrix = nx.to_numpy_array(G)
    
    # Create feature matrix (each row contains the x,y coordinates of a point)
    feature_matrix = np.array([G.nodes[i]['pos'] for i in range(len(points))])
    
    return G, adj_matrix, feature_matrix



def cluster_with_single_majority(to_cluster, eps=10, min_samples=2):
    """
    Clusters data, identifying only one majority cluster and marking all other points as outliers.

    Args:
        to_cluster: List of data points to cluster.
        eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
        min_samples: The number of samples in a neighborhood for a point to be considered as a core point.

    Returns:
        NumPy array of labels, where the majority cluster is labeled 0, and all other points are labeled -1.
    """
    to_cluster_array = np.array(to_cluster)

    if len(to_cluster_array) == 0:
      return np.array([])

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(to_cluster_array)

    # Count the occurrences of each label
    label_counts = Counter(labels)

    # Find the majority cluster label (excluding -1 outliers)
    majority_label = None
    max_count = 0
    for label, count in label_counts.items():
        if label != -1 and count > max_count:
            majority_label = label
            max_count = count

    # Create a new label array where the majority cluster is 0 and all others are -1
    new_labels = np.full(len(labels), -1)  # Initialize all as outliers

    if majority_label is not None:
        new_labels[labels == majority_label] = 0  # Assign 0 to the majority cluster

    return new_labels



In [3]:
def generate_graph(pg_no, height, bg_img, viz=False):
    # Get unique prefixes efficiently without sorting everything
    for entry in os.scandir(DATA_PATH):
        # if entry.is_file() and entry.name.endswith(f'{pg_no}.txt'):
        if entry.is_file() and entry.name == f'{pg_no}_points.txt':

            bg_img = plt.imread(IMG_PATH+f"/{pg_no}.jpg")  # Replace with your image path
            bg_img = cv2.resize(bg_img, (bg_img.shape[1] // 2, bg_img.shape[0] // 2))
            _height, width = bg_img.shape[:2]
            assert(_height == height)

            points_file = entry.path
            print(f"Processing file: {points_file}")
            points = np.loadtxt(points_file)
            points[:, 1] = height - points[:, 1]
            
            # Build a KD-tree for fast neighbor lookup
            tree = cKDTree(points)
            _, indices = tree.query(points, k=NUM_NEIGHBOURS)
            
            segments = []
            seg_colors = [] 

            to_cluster = []
            to_plot = []

            # Process nearest neighbors
            for current_point_index, nbr_indices in enumerate(indices):
                normalized_points = points[nbr_indices] - points[current_point_index]
                scaling_factor = np.max(np.abs(normalized_points))
                if scaling_factor == 0:
                    scaling_factor = 1
                scaled_points = normalized_points / scaling_factor
                relative_neighbours = [(global_idx, sp, np)
                                    for global_idx, sp, np in zip(nbr_indices, scaled_points, normalized_points)]
                

                filtered_neighbours = []
                for _a, _b in combinations(relative_neighbours, 2):
                    # calculate the angle between the two pair vectors
                    if np.linalg.norm(_a[1]) * np.linalg.norm(_b[1]) == 0:
                        cos_similarity = 0.0
                    else:
                        cos_similarity = np.dot(_a[1], _b[1]) / (np.linalg.norm(_a[1]) * np.linalg.norm(_b[1]))

                    # calculate the non-normalized length of the pair
                    norm1 = np.linalg.norm(_a[2])
                    norm2 = np.linalg.norm(_b[2])
                    total_length = norm1 + norm2

                    # select pairs who have an angle of around 180 degrees between them
                    if cos_similarity < cos_similarity_less_than:
                        filtered_neighbours.append((_a, _b, total_length, cos_similarity))

                if filtered_neighbours:
                    # select the pair with the shortest total_length (sometimes, the 2nd shortest is the right answer)
                    shortest_pair = min(filtered_neighbours, key=lambda x: x[2])

                    _connection_1, _connection_2, total_length, cos_similarity = shortest_pair
                    global_idx_connection_1 = _connection_1[0]
                    global_idx_connection_2 = _connection_2[0]

                    # calculate the angles made with the X-axis for clustering purpose
                    theta_a = np.degrees(np.arctan2(_connection_1[2][1], _connection_1[2][0]))
                    theta_b = np.degrees(np.arctan2(_connection_2[2][1], _connection_2[2][0]))
                    theta = (theta_a,theta_b)
                    
                    to_plot.append((current_point_index, global_idx_connection_1, global_idx_connection_2))
                    
                    # Add new informative features for improved clustering
                    y_diff1 = abs(_connection_1[2][1])  # Vertical distance component
                    y_diff2 = abs(_connection_2[2][1])
                    avg_y_diff = (y_diff1 + y_diff2) / 2
                    
                    x_diff1 = abs(_connection_1[2][0])  # Horizontal distance component
                    x_diff2 = abs(_connection_2[2][0])
                    avg_x_diff = (x_diff1 + x_diff2) / 2
                    
                    # Calculate aspect ratio of the connection (height/width)
                    aspect_ratio = avg_y_diff / max(avg_x_diff, 0.001)  # Avoid division by zero
                    
                    # Calculate vertical alignment consistency (low value = better alignment)
                    vert_consistency = abs(y_diff1 - y_diff2)
                    
                    to_cluster.append((
                        total_length, 
                        np.abs(theta[0]+theta[1]),
                        aspect_ratio,
                        vert_consistency,
                        avg_y_diff  # Vertical distance is important for line detection
                    ))


            to_cluster_array = np.array(to_cluster)  # Convert list to NumPy array
            labels = cluster_with_single_majority(to_cluster)



            # removing red (outliers detected by clustering)
            mask = labels != -1
            # Filter labels using the mask
            filtered_labels = labels[mask]
            # Filter to_plot using the same mask
            filtered_to_plot = [item for i, item in enumerate(to_plot) if mask[i]]

            # Create the graph and matrices
            G_generated, adjacency_matrix, feature_matrix = convert_to_graph(filtered_to_plot , points)
            # thus we have a graph, where nodes represent the presence of a character, and characters(nodes) who are part of the same line have edges (are connected) to previous and next characters
            
            if viz == True:
                # Get node positions and labels from the generated graph
                pos = nx.get_node_attributes(G_generated, 'pos')
                node_labels = nx.get_node_attributes(G_generated, 'label')
                unique_labels = sorted(set(node_labels.values()))
                
                # Use a soft, colorblind-friendly palette (e.g., seaborn's "colorblind" palette)
                import seaborn as sns
                palette = sns.color_palette("colorblind", len(unique_labels))
                label_to_color = {label: palette[i] for i, label in enumerate(unique_labels)}
                node_colors = [label_to_color[node_labels[node]] for node in G_generated.nodes()]
                
                # Set up the figure and axes with the background image
                plt.figure(figsize=(16, 8))
                ax = plt.gca()
                plt.imshow(bg_img, alpha=0.4, zorder=0, extent=[0, width, 0, height])
                ax.set_xlim(0, width)
                ax.set_ylim(0, height)
                
                # Draw the graph edges and nodes (small nodes)
                nx.draw_networkx_edges(G_generated, pos, alpha=0.5, ax=ax)
                nx.draw_networkx_nodes(G_generated, pos, node_color=node_colors, node_size=5, ax=ax)
                # nx.draw_networkx_labels(G_generated, pos, font_size=8, ax=ax)
                
                # Draw a transparent bounding box around each node, using the same color as the node label
                box_size = 30  # Adjust the box size as needed
                for node, (x, y) in pos.items():
                    lower_left_x = x - box_size / 2
                    lower_left_y = y - box_size / 2
                    rect = plt.Rectangle((lower_left_x, lower_left_y), box_size, box_size,
                                        linewidth=1,
                                        edgecolor='none',
                                        facecolor=label_to_color[node_labels[node]],
                                        alpha=0.1)
                    ax.add_patch(rect)
                
                plt.title("Graph Visualization with Label Colors and Bounding Boxes")
                plt.axis("off")
                plt.show()


            return G_generated

def get_ground_truth_graph(pg_no, height):
    # Read the files
    points = np.loadtxt(DATA_PATH+f'/{pg_no}_points.txt')
    labels = np.loadtxt(DATA_PATH+f'/{pg_no}_labels.txt', dtype=int)
    points[:, 1] = height - points[:, 1]

    # Create an empty graph for the ground truth
    G_truth = nx.Graph()

    # Add nodes with their positions and labels
    for i, (point, label) in enumerate(zip(points, labels)):
        G_truth.add_node(i, pos=tuple(point), label=label)

    # For each label, connect all points using the Minimum Spanning Tree (MST)
    unique_labels = np.unique(labels)
    for lab in unique_labels:
        # Get all indices for the current label
        indices = [i for i, l in enumerate(labels) if l == lab]
        if len(indices) > 1:
            # Create a complete graph for these nodes with edge weights as Euclidean distances
            H = nx.Graph()
            for i in indices:
                H.add_node(i, pos=G_truth.nodes[i]['pos'])
            for idx, i in enumerate(indices):
                for j in indices[idx+1:]:
                    p1 = G_truth.nodes[i]['pos']
                    p2 = G_truth.nodes[j]['pos']
                    dist = sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)
                    H.add_edge(i, j, weight=dist)
            # Compute the MST for this group, which connects all points with the minimum required edges
            MST = nx.minimum_spanning_tree(H)
            # Add the MST edges to the ground truth graph
            for edge in MST.edges:
                G_truth.add_edge(*edge)

    print("Ground truth graph edges:", list(G_truth.edges))



    return G_truth


def evaluate(G_truth, G_generated):
    # Compute edge sets as sorted tuples for consistency
    edges_truth = {tuple(sorted(edge)) for edge in G_truth.edges}
    edges_generated = {tuple(sorted(edge)) for edge in G_generated.edges}

    correct_edges = edges_truth & edges_generated
    extra_edges = edges_generated - edges_truth
    missing_edges = edges_truth - edges_generated

    print("Correct edges:", len(correct_edges))
    print("Extra edges (wrong):", len(extra_edges))
    print("Missing edges:", len(missing_edges))

    # Ensure each node has a label; if not, assign a default value (e.g., -1)
    nodes_truth = sorted(G_truth.nodes(data=True), key=lambda x: x[0])
    nodes_generated = sorted(G_generated.nodes(data=True), key=lambda x: x[0])

    true_labels = [data.get('label') if data.get('label') is not None else -1 for _, data in nodes_truth]
    generated_labels = [data.get('label') if data.get('label') is not None else -1 for _, data in nodes_generated]

    ari = adjusted_rand_score(true_labels, generated_labels)
    vmeasure = v_measure_score(true_labels, generated_labels)


    ari = adjusted_rand_score(true_labels, generated_labels)
    vmeasure = v_measure_score(true_labels, generated_labels)

    print("Adjusted Rand Index:", ari)
    print("V-measure:", vmeasure)

    return (len(correct_edges), len(extra_edges), len(missing_edges), ari, vmeasure)


In [4]:
# pg_no = 0
# bg_img = plt.imread(IMG_PATH+f"/{pg_no}.jpg")  # Replace with your image path
# bg_img = cv2.resize(bg_img, (bg_img.shape[1] // 2, bg_img.shape[0] // 2))
# height, width = bg_img.shape[:2]

# # Generate and evaluate graphs
# G_generated = generate_graph(pg_no, height,bg_img,viz=True)

In [5]:
# Initialize empty lists to store results
results = []

# Process all 14 pages
for pg_no in range(14):
    # Load and resize background image
    bg_img = plt.imread(IMG_PATH+f"/{pg_no}.jpg")  # Replace with your image path
    bg_img = cv2.resize(bg_img, (bg_img.shape[1] // 2, bg_img.shape[0] // 2))
    height, width = bg_img.shape[:2]
    
    # Generate and evaluate graphs
    G_generated = generate_graph(pg_no, height, bg_img)
    G_truth = get_ground_truth_graph(pg_no, height)
    correct_edges, extra_edges, missing_edges, ari, vmeasure = evaluate(G_truth, G_generated)
    
    # Calculate metrics
    total_truth_edges = len(G_truth.edges())
    total_generated_edges = len(G_generated.edges())
    
    precision = correct_edges / total_generated_edges if total_generated_edges > 0 else 0
    recall = correct_edges / total_truth_edges if total_truth_edges > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    # Append results to list
    results.append({
        'Page': pg_no,
        'Correct Edges': correct_edges,
        'Extra Edges': extra_edges,
        'Missing Edges': missing_edges,
        'Total Truth Edges': total_truth_edges,
        'Total Generated Edges': total_generated_edges,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ari':ari,
        'vmeasure': vmeasure
    })
    
    # Optional: Print progress
    print(f"Processed page {pg_no}")

# Create pandas DataFrame from results
results_df = pd.DataFrame(results)

# Calculate summary statistics
summary = {
    'Page': 'Summary',
    'Correct Edges': results_df['Correct Edges'].sum(),
    'Extra Edges': results_df['Extra Edges'].sum(),
    'Missing Edges': results_df['Missing Edges'].sum(),
    'Total Truth Edges': results_df['Total Truth Edges'].sum(),
    'Total Generated Edges': results_df['Total Generated Edges'].sum(),
    'Precision': results_df['Correct Edges'].sum() / results_df['Total Generated Edges'].sum() if results_df['Total Generated Edges'].sum() > 0 else 0,
    'Recall': results_df['Correct Edges'].sum() / results_df['Total Truth Edges'].sum() if results_df['Total Truth Edges'].sum() > 0 else 0,
    'F1 Score': np.nan,  # Will be calculated below
    'ari': results_df['ari'].mean(),
    'vmeasure': results_df['vmeasure'].mean(),
}

# Calculate overall F1 score
summary['F1 Score'] = 2 * summary['Precision'] * summary['Recall'] / (summary['Precision'] + summary['Recall']) if (summary['Precision'] + summary['Recall']) > 0 else 0

# Add summary row to DataFrame
results_df = pd.concat([results_df, pd.DataFrame([summary])], ignore_index=True)

# Display the results DataFrame
print("\nEvaluation Results:")
print(results_df.to_string(index=False))

# Optionally, format the floating-point columns for better readability
formatted_df = results_df.copy()
for col in ['Precision', 'Recall', 'F1 Score']:
    formatted_df[col] = formatted_df[col].map('{:.4f}'.format)

print("\nFormatted Evaluation Results:")
print(formatted_df.to_string(index=False))

Processing file: /mnt/cai-data/manuscript-annotation-tool/manuscripts/eeml-submission/points-2D/0_points.txt
Ground truth graph edges: [(0, 1), (1, 2), (2, 3), (3, 4), (4, 7), (5, 8), (5, 9), (6, 10), (6, 11), (7, 31), (8, 41), (9, 10), (11, 42), (12, 13), (12, 43), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 44), (21, 45), (21, 46), (22, 48), (22, 47), (23, 24), (23, 48), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (31, 32), (32, 33), (33, 34), (34, 35), (35, 49), (36, 37), (36, 50), (37, 38), (38, 51), (39, 53), (39, 40), (40, 41), (42, 43), (44, 45), (46, 47), (49, 50), (51, 52), (52, 53), (54, 61), (54, 55), (55, 56), (56, 57), (57, 62), (58, 59), (59, 60), (60, 61), (62, 63), (63, 65), (64, 65), (66, 67), (67, 68), (68, 69), (69, 70), (70, 87), (71, 87), (71, 88), (72, 89), (72, 73), (73, 90), (74, 101), (74, 100), (75, 76), (75, 103), (76, 77), (77, 104), (78, 106), (78, 107), (79, 112), (79, 111), (80, 81), (80, 112), (81, 82), (82,

## Seamformer

In [6]:
# helper functions

def point_in_polygon(point, polygon):
    """Check if a point is inside a polygon using matplotlib's Path."""
    path = Path(polygon)
    return path.contains_point(point)

def label_points(points, polygons):
    """
    Label each point based on which polygon contains it.
    Returns:
    - labels: array of same length as points, with values indicating polygon index
              -1 indicates point is not in any polygon
    """
    labels = np.full(len(points), -1)  # Default: not in any polygon
    
    for i, polygon in enumerate(polygons):
        # Convert polygon to numpy array if it's not already
        poly_array = np.array(polygon)
        
        # Create path for efficient checking
        path = Path(poly_array)
        
        # Check all points at once for this polygon
        inside_mask = path.contains_points(points)
        
        # Only set labels for points not already labeled
        # This handles potential overlapping polygons - first polygon gets priority
        labels[np.logical_and(inside_mask, labels == -1)] = i
    
    return labels

def generate_distinct_colors(n_colors):
    """Generate visually distinct colors."""
    colors = []
    for i in range(n_colors):
        # Use HSV color space for more distinct colors
        hue = i / n_colors
        saturation = 0.8 + random.uniform(-0.2, 0.2)  # Add slight randomness
        value = 0.8 + random.uniform(-0.2, 0.2)
        colors.append(colorsys.hsv_to_rgb(hue, saturation, value))
    return colors

def visualize_labeled_points(points, polygons, labels, bg_img):
    """Visualize the points colored by label and the polygons."""
    fig, ax = plt.subplots(figsize=(12, 10))
    height, width = bg_img.shape[:2]
    plt.imshow(bg_img, alpha=0.4, zorder=0, extent=[0, width, 0, height])
    ax.set_xlim(0, width)  # Use ax.set_xlim instead of plt.set_xlim
    ax.set_ylim(0, height)  # Use ax.set_ylim instead of plt.set_ylim
    
    # Count number of unique labels (excluding -1)
    unique_labels = np.unique(labels)
    positive_labels = unique_labels[unique_labels >= 0]
    n_labels = len(positive_labels)
    
    # Generate colors for each label and polygon
    # Add one more color for unlabeled points
    colors = generate_distinct_colors(n_labels + 1)
    
    # Draw polygons with transparency
    for i, polygon in enumerate(polygons):
        if i < n_labels:  # Ensure we don't go beyond available colors
            poly = MatplotlibPolygon(np.array(polygon), 
                                     alpha=0.3, 
                                     fill=True, 
                                     color=colors[i],
                                     label=f'Polygon {i}')
            ax.add_patch(poly)
    
    # Plot points, colored by their label
    for label in unique_labels:
        mask = labels == label
        if label == -1:
            # Use the last color for unlabeled points
            ax.scatter(points[mask, 0], points[mask, 1], color='gray', 
                       alpha=0.7, label='Unlabeled')
        else:
            ax.scatter(points[mask, 0], points[mask, 1], color=colors[int(label)], 
                       alpha=0.7, label=f'Label {label}')
    
    ax.set_title('Points Labeled by Containing Polygon')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1))
    ax.grid(True, linestyle='--', alpha=0.6)
    
    plt.tight_layout()
    return fig

def scale_polygons(polygons, scale_factor=0.5):
    """Scale all polygons by the given factor (default: 0.5 for scaling down by 2)"""
    scaled_polygons = []
    
    for polygon in polygons:
        # Convert to numpy array for easier manipulation
        poly_array = np.array(polygon)
        
        # Scale coordinates
        scaled_poly = poly_array * scale_factor
        
        # Convert back to list format
        scaled_polygons.append(scaled_poly.tolist())
    
    return scaled_polygons

def invert_polygons(polygons, height):
    """Scale all polygons by the given factor (default: 0.5 for scaling down by 2)"""
    inverted_polygons = []
    
    for polygon in polygons:
        # Convert to numpy array for easier manipulation
        poly_array = np.array(polygon)
        
        # invert coordinates
        poly_array[:, 1] = height - poly_array[:, 1]
        
        # Convert back to list format
        inverted_polygons.append(poly_array.tolist())
    
    return inverted_polygons


In [7]:
# Open and load the JSON file
with open(SEAMFORMER_RESULTS, "r", encoding="utf-8") as file:
    data = json.load(file)


In [ ]:
def get_seamformer_graph(idx,data,bg_img,viz=False):
    # Read the files
    pg_no = data[idx]['imgPath'].split("/")[-1].split('.')[0]
    points = np.loadtxt(DATA_PATH+f'/{pg_no}_points.txt')

    
    bounding_polygons = data[idx]['predPolygons'] # A list of list(polygons) of list(2D coordinates)
    scaled_bounding_polygons = scale_polygons(bounding_polygons) # scale down by 2

    # Label the points
    labels = label_points(points, scaled_bounding_polygons)
    _height = int(data[idx]['imgDims'][0]/2) #scale down by 2
    points[:, 1] = _height - points[:, 1]
    # scaled_bounding


    if viz == True:
        inverted_polygons = invert_polygons(scaled_bounding_polygons,_height)
        fig = visualize_labeled_points(points, inverted_polygons, labels, bg_img)
        ax = plt.gca()  # Get current axes
        # plt.gca().invert_yaxis()  # Flip the y-axis
        plt.show()

    # Remove points with label -1
    # mask = labels != -1
    # points = points[mask]
    # labels = labels[mask]


    


    # Create an empty graph for the ground truth
    G_truth = nx.Graph()

    # Add nodes with their positions and labels
    for i, (point, label) in enumerate(zip(points, labels)):
        G_truth.add_node(i, pos=tuple(point), label=label)

    # For each label, connect all points using the Minimum Spanning Tree (MST)
    unique_labels = np.unique(labels)
    for lab in unique_labels:
        if lab!=-1: #IMPORTANT = we do not apply algorithm on undefined points..
            # Get all indices for the current label
            indices = [i for i, l in enumerate(labels) if l == lab]
            if len(indices) > 1:
                # Create a complete graph for these nodes with edge weights as Euclidean distances
                H = nx.Graph()
                for i in indices:
                    H.add_node(i, pos=G_truth.nodes[i]['pos'])
                for idx, i in enumerate(indices):
                    for j in indices[idx+1:]:
                        p1 = G_truth.nodes[i]['pos']
                        p2 = G_truth.nodes[j]['pos']
                        dist = sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)
                        H.add_edge(i, j, weight=dist)
                # Compute the MST for this group, which connects all points with the minimum required edges
                MST = nx.minimum_spanning_tree(H)
                # Add the MST edges to the ground truth graph
                for edge in MST.edges:
                    G_truth.add_edge(*edge)

    print("Seamformer graph edges:", list(G_truth.edges))
    if viz == True:
        # Visualize the ground truth graph
        pos = nx.get_node_attributes(G_truth, 'pos')
        plt.figure(figsize=(8, 6))
        height, width = bg_img.shape[:2]
        plt.imshow(bg_img, alpha=0.4, zorder=0, extent=[0, width, 0, height])
        nx.draw(G_truth, pos, with_labels=True, node_color='lightblue', edge_color='green', node_size=10, font_size=3)
        plt.title("Ground Truth Graph (via MST per Label Group)")
        plt.axis('equal')
        plt.show()



    return G_truth, pg_no


def get_ground_truth_graph_for_seamformer(pg_no,data,idx,big_img, viz = False):
    # Read the files
    points = np.loadtxt(DATA_PATH+f'/{pg_no}_points.txt')
    labels = np.loadtxt(DATA_PATH+f'/{pg_no}_labels.txt', dtype=int)
    height = int(data[idx]['imgDims'][0]/2) #scale down by 2
    points[:, 1] = height - points[:, 1]

    # Create an empty graph for the ground truth
    G_truth = nx.Graph()

    # Add nodes with their positions and labels
    for i, (point, label) in enumerate(zip(points, labels)):
        G_truth.add_node(i, pos=tuple(point), label=label)

    # For each label, connect all points using the Minimum Spanning Tree (MST)
    unique_labels = np.unique(labels)
    for lab in unique_labels:
        # Get all indices for the current label
        indices = [i for i, l in enumerate(labels) if l == lab]
        if len(indices) > 1:
            # Create a complete graph for these nodes with edge weights as Euclidean distances
            H = nx.Graph()
            for i in indices:
                H.add_node(i, pos=G_truth.nodes[i]['pos'])
            for idx, i in enumerate(indices):
                for j in indices[idx+1:]:
                    p1 = G_truth.nodes[i]['pos']
                    p2 = G_truth.nodes[j]['pos']
                    dist = sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)
                    H.add_edge(i, j, weight=dist)
            # Compute the MST for this group, which connects all points with the minimum required edges
            MST = nx.minimum_spanning_tree(H)
            # Add the MST edges to the ground truth graph
            for edge in MST.edges:

                G_truth.add_edge(*edge)

    print("Groundtruth graph edges:", list(G_truth.edges))
    if viz == True:
        # Visualize the ground truth graph
        pos = nx.get_node_attributes(G_truth, 'pos')

        plt.figure(figsize=(8, 6))
        nx.draw(G_truth, pos, with_labels=True, node_color='lightblue', edge_color='green', node_size=10, font_size=3)
        plt.title("Ground Truth Graph (via MST per Label Group)")
        ax = plt.gca()  # Get current axes
        height, width = bg_img.shape[:2]
        plt.imshow(bg_img, alpha=0.4, zorder=0, extent=[0, width, 0, height])
        ax.set_xlim(0, width)  # Use ax.set_xlim instead of plt.set_xlim
        ax.set_ylim(0, height)  # Use ax.set_ylim instead of plt.set_ylim


        plt.axis('equal')
        plt.show()

    print("Ground truth graph edges:", list(G_truth.edges))

    return G_truth



In [9]:
def evaluate_for_seamformer(G_truth, G_generated,bg_img, viz = False):
    # Compute edge sets as sorted tuples for consistency
    edges_truth = {tuple(sorted(edge)) for edge in G_truth.edges}
    edges_generated = {tuple(sorted(edge)) for edge in G_generated.edges}

    correct_edges = edges_truth & edges_generated
    extra_edges = edges_generated - edges_truth
    missing_edges = edges_truth - edges_generated

    print("Correct edges:", len(correct_edges))
    print("Extra edges (wrong):", len(extra_edges))
    print("Missing edges:", len(missing_edges))

    if viz == True:
        # Use positions from the ground truth graph (assuming nodes have 'pos' attribute)
        pos = nx.get_node_attributes(G_truth, 'pos')

        plt.figure(figsize=(12, 6))
        ax = plt.gca()  # Get current axes
        height, width = bg_img.shape[:2]
        plt.imshow(bg_img, alpha=0.4, zorder=0, extent=[0, width, 0, height])
        ax.set_xlim(0, width)  # Use ax.set_xlim instead of plt.set_xlim
        ax.set_ylim(0, height)  # Use ax.set_ylim instead of plt.set_ylim

        # Draw nodes
        nx.draw_networkx_nodes(G_truth, pos, node_color='white', node_size=18)

        # Draw correct edges in blue-green (teal)
        nx.draw_networkx_edges(G_truth, pos, edgelist=list(correct_edges), edge_color='#009E73', width=4, label='Correct')

        # Draw extra edges in orange
        if extra_edges:
            nx.draw_networkx_edges(G_generated, pos, edgelist=list(extra_edges), edge_color='#E69F00', width=8,alpha=0.5, label='Extra (Wrong)')

        # Draw missing edges in purple
        if missing_edges:
            nx.draw_networkx_edges(G_truth, pos, edgelist=list(missing_edges), edge_color='#CC79A7', width=4, label='Missing')

        # Draw node labels
        nx.draw_networkx_labels(G_truth, pos, font_size=1, font_color='black')

        # Lines for edge types in the legend
        correct_line = mlines.Line2D([], [], color='#009E73', lw=2, label='Correct Edge')
        extra_line = mlines.Line2D([], [], color='#E69F00', lw=2,  label='Extra Edge')
        missing_line = mlines.Line2D([], [], color='#CC79A7', lw=2,  label='Missing Edge')

        # White dot with black edge for character presence
        character_marker = mlines.Line2D([], [], color='black', marker='o', linestyle='None',
                                        markerfacecolor='white', markeredgecolor='black', 
                                        markersize=10, label='Node (Character)')

        # Add legend
        plt.legend(handles=[correct_line, extra_line, missing_line, character_marker], loc='best')

        # plt.title("Evaluation: Correct (green), Extra (orange), Missing (blue)")
        plt.axis('off')
        plt.show()

    nodes_truth = sorted(G_truth.nodes(data=True), key=lambda x: x[0])
    nodes_generated = sorted(G_generated.nodes(data=True), key=lambda x: x[0])

    true_labels = [data.get('label') for _, data in nodes_truth]
    generated_labels = [data.get('label') for _, data in nodes_generated]

    ari = adjusted_rand_score(true_labels, generated_labels)
    vmeasure = v_measure_score(true_labels, generated_labels)

    print("Adjusted Rand Index:", ari)
    print("V-measure:", vmeasure)

    return (len(correct_edges), len(extra_edges), len(missing_edges), ari, vmeasure)

In [10]:
# visualizing the seamformer segmentation results (can skip)
# idx = 13 #4,13
# pg_no = data[idx]['imgPath'].split("/")[-1].split('.')[0]
# bg_img = plt.imread(IMG_PATH+f"/{pg_no}.jpg")  # Replace with your image path
# bg_img = cv2.resize(bg_img, (bg_img.shape[1] // 2, bg_img.shape[0] // 2))

# G_seamformer,pg_no = get_seamformer_graph(idx,data,bg_img, viz = True)
# G_truth = get_ground_truth_graph_for_seamformer(pg_no,data,idx,bg_img, viz = True)
# evaluate_for_seamformer(G_truth,G_seamformer,bg_img, viz = True)

In [11]:
# Initialize empty lists to store results
results = []

# Process all 14 pages
for idx in range(15):
    bg_img = plt.imread(IMG_PATH+f"/{pg_no}.jpg")  # Replace with your image path
    bg_img = cv2.resize(bg_img, (bg_img.shape[1] // 2, bg_img.shape[0] // 2))
    height, width = bg_img.shape[:2]

    G_seamformer,pg_no = get_seamformer_graph(idx,data,bg_img)
    print(pg_no)
    G_truth = get_ground_truth_graph_for_seamformer(pg_no,data,idx,bg_img)
    # Load and resize background image

    
    correct_edges, extra_edges, missing_edges, ari, vmeasure = evaluate_for_seamformer(G_truth, G_seamformer, bg_img)
    
    # Calculate metrics
    total_truth_edges = len(G_truth.edges())
    total_generated_edges = len(G_seamformer.edges())
    
    precision = correct_edges / total_generated_edges if total_generated_edges > 0 else 0
    recall = correct_edges / total_truth_edges if total_truth_edges > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    # Append results to list
    results.append({
        'Page': pg_no,
        'Correct Edges': correct_edges,
        'Extra Edges': extra_edges,
        'Missing Edges': missing_edges,
        'Total Truth Edges': total_truth_edges,
        'Total Generated Edges': total_generated_edges,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ari': ari,
        'vmeasure': vmeasure
    })
    
    # Optional: Print progress
    print(f"Processed page {pg_no}")

# Create pandas DataFrame from results
results_df = pd.DataFrame(results)

# Calculate summary statistics
summary = {
    'Page': 'Summary',
    'Correct Edges': results_df['Correct Edges'].sum(),
    'Extra Edges': results_df['Extra Edges'].sum(),
    'Missing Edges': results_df['Missing Edges'].sum(),
    'Total Truth Edges': results_df['Total Truth Edges'].sum(),
    'Total Generated Edges': results_df['Total Generated Edges'].sum(),
    'Precision': results_df['Correct Edges'].sum() / results_df['Total Generated Edges'].sum() if results_df['Total Generated Edges'].sum() > 0 else 0,
    'Recall': results_df['Correct Edges'].sum() / results_df['Total Truth Edges'].sum() if results_df['Total Truth Edges'].sum() > 0 else 0,
    'F1 Score': np.nan,  # Will be calculated below
    'ari': results_df['ari'].mean(),
    'vmeasure': results_df['vmeasure'].mean()
}

# Calculate overall F1 score
summary['F1 Score'] = 2 * summary['Precision'] * summary['Recall'] / (summary['Precision'] + summary['Recall']) if (summary['Precision'] + summary['Recall']) > 0 else 0

# Add summary row to DataFrame
results_df = pd.concat([results_df, pd.DataFrame([summary])], ignore_index=True)

# Display the results DataFrame
print("\nEvaluation Results:")
print(results_df.to_string(index=False))

# Optionally, format the floating-point columns for better readability
formatted_df = results_df.copy()
for col in ['Precision', 'Recall', 'F1 Score']:
    formatted_df[col] = formatted_df[col].map('{:.4f}'.format)

print("\nFormatted Evaluation Results:")
print(formatted_df.to_string(index=False))

Seamformer graph edges: [(0, 5), (0, 1), (2, 4), (2, 5), (3, 4), (7, 13), (7, 42), (8, 21), (8, 10), (9, 16), (9, 32), (10, 20), (11, 23), (11, 48), (12, 52), (12, 25), (13, 41), (14, 27), (14, 28), (15, 31), (15, 16), (17, 18), (17, 33), (18, 19), (19, 20), (21, 47), (22, 47), (22, 34), (23, 35), (24, 38), (24, 39), (25, 41), (26, 27), (28, 43), (29, 43), (29, 30), (30, 44), (31, 46), (32, 33), (34, 48), (35, 36), (36, 49), (37, 49), (37, 50), (38, 51), (39, 40), (40, 52), (42, 53), (44, 45), (45, 46), (50, 51), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (62, 65), (62, 84), (63, 87), (63, 66), (64, 69), (64, 92), (65, 83), (66, 88), (67, 68), (67, 91), (68, 107), (69, 109), (70, 93), (70, 71), (71, 72), (72, 94), (73, 94), (73, 74), (74, 75), (75, 95), (76, 77), (76, 95), (77, 96), (78, 79), (78, 96), (79, 80), (80, 81), (81, 82), (82, 83), (84, 97), (85, 99), (85, 98), (86, 101), (86, 87), (88, 89), (89, 102), (90, 105), (90, 106), (91, 106), (92,